In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [3]:
train.shape

(19579, 3)

In [4]:
train[:5]

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [5]:
from sklearn.feature_extraction.text import CountVectorizer

dataset = train.append(test)
dataset.shape
word_dict = CountVectorizer().fit(dataset["text"])
dict_size = len(word_dict.vocabulary_)
dict_size

28300

In [6]:
import numpy as np

np.array(list(word_dict.vocabulary_.values())).min()

0

In [7]:
x_train = word_dict.transform(train["text"])
x_train.shape

(19579, 28300)

In [8]:
train["author"] = train["author"].replace("EAP",0)
train["author"] = train["author"].replace("HPL",1)
train["author"] = train["author"].replace("MWS",2)

from sklearn.preprocessing import OneHotEncoder

y_train = OneHotEncoder().fit_transform(train["author"].as_matrix().reshape(-1, 1))
y_train.shape

(19579, 3)

In [9]:
x_train = x_train.todense()

In [10]:
y_train = y_train.todense()

In [11]:
import keras
from keras.layers import Dense, Embedding, Bidirectional, Input, LSTM, Bidirectional
from keras.models import Model, Sequential
from keras.layers.recurrent import SimpleRNN
from keras.layers.wrappers import TimeDistributed
from keras.layers import Dropout, Reshape, Flatten

Using TensorFlow backend.


In [12]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15712642803272587156
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5586747392
locality {
  bus_id: 1
}
incarnation: 6441135987058405704
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [59]:
model = Sequential([
    Dense(16, input_shape=(dict_size,), activation="tanh"),
    Dense(32, activation="tanh"),
    Dense(3, activation="softmax")
])

In [60]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [61]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 16)                452816    
_________________________________________________________________
dense_35 (Dense)             (None, 32)                544       
_________________________________________________________________
dense_36 (Dense)             (None, 3)                 99        
Total params: 453,459
Trainable params: 453,459
Non-trainable params: 0
_________________________________________________________________


In [17]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=0)

In [63]:
model.fit(x_train, y_train, epochs=4, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Train on 15663 samples, validate on 3916 samples
Epoch 1/4
15663/15663 [==============================] - 3s 219us/step - loss: 0.5727 - acc: 0.7687 - val_loss: 0.4180 - val_acc: 0.8294
Epoch 2/4
15663/15663 [==============================] - 3s 194us/step - loss: 0.1986 - acc: 0.9257 - val_loss: 0.4698 - val_acc: 0.8292


А сега нека да пробваме и lstm layer

In [13]:
model = Sequential([
    Embedding(dict_size,128, input_shape=(dict_size,)),
    LSTM(128),
    Dense(3, activation="softmax")
])

In [14]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28300, 128)        3622400   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 3,754,371
Trainable params: 3,754,371
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(x_train, y_train, epochs=4, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Train on 15663 samples, validate on 3916 samples
Epoch 1/4
 1696/15663 [==>...........................] - ETA: 3:27:11 - loss: 1.0872 - acc: 0.4098

KeyboardInterrupt: 